<a href="https://colab.research.google.com/github/AJH0625/DeZero/blob/main/%EC%A0%9C2_%EA%B3%A0%EC%A7%80_%EC%9E%90%EC%97%B0%EC%8A%A4%EB%9F%AC%EC%9A%B4_%EC%BD%94%EB%93%9C%EB%A1%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##11.1 Function 클래스 수정

In [7]:
class Function:
  def __call__(self, inputs):
    xs = [x.data for x in inputs]
    ys = self.forward(xs)
    outputs = [Variable(as_array(y)) for y in ys]

    for output in outputs:
      output.set_creator(self)
    self.inputs = inputs
    self.outputs = outputs
    return outputs

  def forward(self, x):
    raise NotImplementedError()

  def backward(self, gy):
    raise NotImplementedError()

##11.2 Add 클래스 구현

In [8]:
class Add(Function):
  def forward(self, xs):
    x0, x1 = xs
    y = x0 + x1
    return (y,)

In [13]:
class Variable:
  def __init__(self, data):
    self.data = data
    self.grad = None
    self.creator = None
  
  def set_creator(self,func):
    self.creator = func

In [11]:
def as_array(x):
  if np.isscalar(x):
    return np.array(x)
  return x

In [14]:
import numpy as np
xs = [Variable(np.array(2)), Variable(np.array(3))]
f = Add()
ys = f(xs)
y = ys[0]
print(y.data)

5


##12.1 첫 번째 개선: 함수를 사용하기 쉽게

In [18]:
class Function:
  def __call__(self, *inputs):
    xs = [x.data for x in inputs]
    ys = self.forward(xs)
    outputs = [Variable(as_array(y)) for y in ys]

    for output in outputs:
      output.set_creator(self)
    self.inputs = inputs
    self.outputs = outputs

    return outputs if len(outputs) > 1 else outputs[0]

In [19]:
def f(*x):
  print(x)

f(1,2,3)

(1, 2, 3)


##12.2 두 번째 개선: 함수를 구현하기 쉽도록

In [22]:
class Function:
  def __call__(self, *inputs):
    xs = [x.data for x in inputs]
    ys = self.forward(*xs)
    if not isinstance(ys, tuple):
      ys = (ys,)
    outputs = [Variable(as_array(y)) for y in ys]

    for output in outputs:
      output.set_creator(self)
    self.inputs = inputs
    self.outputs = outputs

    return outputs if len(outputs) > 1 else outputs[0]

In [23]:
class Add(Function):
  def forward(self, x0, x1):
    y = x0 + x1
    return y

##12.3 add 함수 구현

In [24]:
def add(x0, x1):
  return Add()(x0, x1)

In [25]:
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
y = add(x0, x1)
print(y.data)

5


##13.1 가변 길이 인수에 대응한 Add 클래스의 역전파

In [26]:
class Add(Function):
  def forward(self, x0, x1):
    y = x0 + x1
    return y

  def backward(self, gy):
    return gy,gy

##13.2 Variable 클래스 수정

In [28]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                x.grad = gx

                if x.creator is not None:
                    funcs.append(x.creator)

##13.3 Square 클래스 구현

In [29]:
class Square(Function):
  def forward(self, x):
    y = x**2
    return y
  
  def backward(self, gy):
    x = self.inputs[0].data
    gx = 2*x*gy
    return gx

In [30]:
def square(x):
  return Square()(x)

In [31]:
x = Variable(np.array(2.0))
y = Variable(np.array(3.0))

z = add(square(x), square(y))
z.backward()

print(z.data)
print(x.grad)
print(y.grad)

13.0
4.0
6.0


##14.1 해결책

In [32]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
              if x.grad is None:
                x.grad = gx
              else:
                x.grad = x.grad+gx

                if x.creator is not None:
                    funcs.append(x.creator)

In [34]:
x = Variable(np.array(3.0))
y = add(x,x)
y.backward()
print(x.grad)

2.0


##14.3 미분값 재설정

In [40]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
      if self.grad is None:
        self.grad = np.ones_like(self.data)

      funcs = [self.creator]
      while funcs:
        f = funcs.pop()
        gys = [output.grad for output in f.outputs]
        gxs = f.backward(*gys)
        if not isinstance(gxs, tuple):
          gxs = (gxs,)

        for x, gx in zip(f.inputs, gxs):
          if x.grad is None:
            x.grad = gx
          else:
            x.grad = x.grad+gx
          if x.creator is not None:
            funcs.append(x.creator)

    def cleargrad(self):
      self.grad = None

In [41]:
x = Variable(np.array(3.0))
y = add(add(x,x),x)
y.backward()
print(x.grad)

3.0


##16.1 세대추가

In [42]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    def backward(self):
      if self.grad is None:
        self.grad = np.ones_like(self.data)

      funcs = [self.creator]
      while funcs:
        f = funcs.pop()
        gys = [output.grad for output in f.outputs]
        gxs = f.backward(*gys)
        if not isinstance(gxs, tuple):
          gxs = (gxs,)

        for x, gx in zip(f.inputs, gxs):
          if x.grad is None:
            x.grad = gx
          else:
            x.grad = x.grad+gx
          if x.creator is not None:
            funcs.append(x.creator)

    def cleargrad(self):
      self.grad = None

In [44]:
class Function:
  def __call__(self, *inputs):
    xs = [x.data for x in inputs]
    ys = self.forward(*xs)
    if not isinstance(ys, tuple):
      ys = (ys,)
    outputs = [Variable(as_array(y)) for y in ys]

    self.generation = max([x.generation for x in inpus])
    for output in outputs:
      output.set_creator(self)
    self.inputs = inputs
    self.outputs = outputs

    return outputs if len(outputs) > 1 else outputs[0]

##16.3 Variable 클래스의 backward

In [47]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    def backward(self):
      if self.grad is None:
        self.grad = np.ones_like(self.data)

      funcs = []
      seen_set = set()

      def add_func(f):
        if f not in seen_set:
          funcs.append(f)
          seen_set.add(f)
          funcs.sort(key=lambda x: x.generation)

      add_func(self.creator)

      while funcs:
        f = funcs.pop()
        gys = [output.grad for output in f.outputs]
        gxs = f.backward(*gys)
        if not isinstance(gxs, tuple):
          gxs = (gxs,)

        for x, gx in zip(f.inputs, gxs):
          if x.grad is None:
            x.grad = gx
          else:
            x.grad = x.grad+gx
          if x.creator is not None:
            add_func(x.creator)

    def cleargrad(self):
      self.grad = None

##17.4 weakref 모듈

In [50]:
import weakref
import numpy as np

a = np.array([1,2,3])
b = weakref.ref(a)

print(b)
print(b())

<weakref at 0x7f55a08782f0; to 'numpy.ndarray' at 0x7f55a081e120>
[1 2 3]


In [51]:
import weakref

class Function:
  def __call__(self, *inputs):
    xs = [x.data for x in inputs]
    ys = self.forward(*xs)
    if not isinstance(ys, tuple):
      ys = (ys,)
    outputs = [Variable(as_array(y)) for y in ys]

    self.generation = max([x.generation for x in inpus])
    for output in outputs:
      output.set_creator(self)
    self.inputs = inputs
    self.outputs = [weakref.ref(output) for output in outputs]

    return outputs if len(outputs) > 1 else outputs[0]

In [52]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    def backward(self):
      if self.grad is None:
        self.grad = np.ones_like(self.data)

      funcs = []
      seen_set = set()

      def add_func(f):
        if f not in seen_set:
          funcs.append(f)
          seen_set.add(f)
          funcs.sort(key=lambda x: x.generation)

      add_func(self.creator)

      while funcs:
        f = funcs.pop()
        gys = [output().grad for output in f.outputs]
        gxs = f.backward(*gys)
        if not isinstance(gxs, tuple):
          gxs = (gxs,)

        for x, gx in zip(f.inputs, gxs):
          if x.grad is None:
            x.grad = gx
          else:
            x.grad = x.grad+gx
          if x.creator is not None:
            add_func(x.creator)

    def cleargrad(self):
      self.grad = None

##18.1 필요없는 미분값 삭제

In [2]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    def backward(self, retain_grad=False):
      if self.grad is None:
        self.grad = np.ones_like(self.data)

      funcs = []
      seen_set = set()

      def add_func(f):
        if f not in seen_set:
          funcs.append(f)
          seen_set.add(f)
          funcs.sort(key=lambda x: x.generation)

      add_func(self.creator)

      while funcs:
        f = funcs.pop()
        gys = [output().grad for output in f.outputs]
        gxs = f.backward(*gys)
        if not isinstance(gxs, tuple):
          gxs = (gxs,)

        for x, gx in zip(f.inputs, gxs):
          if x.grad is None:
            x.grad = gx
          else:
            x.grad = x.grad+gx
            if x.creator is not None:
              add_func(x.creator)

          if not retain_grad:
            for y in f.outputs:
              y().grad = None

    def cleargrad(self):
      self.grad = None